$$Exercise: 2$$

#Q-1

$Firstly,\; we\; set\; the \;co-ordinates \;of\; house\; as\;(0,0)\; and \;then\; we\; proceed\; as \;follows- \\
The\; mathematical\; model \;is\; to \;minimize\; the\; total \;distance \;from\;farm\; to\;the\; house\; is: \\ 
Declaring\; the \;decision\; variables: \\ 
x_f = The\; x\; coordinate\; of\; end \;point\; of \;the\; pipe \;in\; farm \\ 
y_f = The\; y \;coordinate \;of\; end \;point\; of\; the\; pipe\; in\; farm \\ 
x_l = The\; x \;coordinate\; of\; end \;point\; of \;the\; pipe\; in \;lake \\ 
y_l = The\; y \;coordinate\; of\; end \;point\; of \;the\; pipe\; in \;lake \\ 
The\; objective\; function\; is: \\ 
{min} (x_f-x_l)^2 + (y_f-y_l)^2 \\
Subject\; to, \\ 
 (x_l-200)^2 +(y_l-200)^2 \leq 2500 \\ 
 (x_f+3y_f)\leq0 \\ 
 0\leq x_f \leq150 \\ 
 -50 \leq y_f \leq 0  $


#Q-2

In [36]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


The above code also fetches an optimization problem in an '.nl' format. 'nl' format is suitable for solvers, not for humans. Lets call ipopt directly (without using any Python) to solve it.

In [37]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ


If you are able to see Ipopt output, we can install pyomo as usual

In [38]:
!pip install -q pyomo

In [39]:
import numpy as np

In [40]:
from pyomo.environ import *

In [41]:
model = ConcreteModel()
model.xf = Var()
model.xl = Var()
model.yf = Var()
model.yl = Var()

model.xf.setlb(0)
model.xf.setub(150)

model.xl.setlb(150)
model.xl.setub(250)

model.yf.setlb(-50)
model.yf.setub(0)

model.yl.setlb(150)
model.yl.setub(250)

model.constraints = ConstraintList()

model.constraints.add(expr= (200-model.xl)**2 + (200-model.yl)**2 <= 2500  )
model.constraints.add(expr= model.xf + 3*model.yf<= 0 )
model.obj = Objective(expr = (model.xl-model.xf)**2 + (model.yl-model.yf)**2, sense=minimize)
model.pprint() 


1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

4 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   150 : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   150 :  None :   250 : False :  True :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   -50 :  None :     0 : False :  True :  Reals
    yl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   150 :  None :   250 : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (xl - xf)**2 + (yl - yf)**2

1 Constraint Declara

In [42]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 4
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.028141021728515625
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



#Q-3

In [43]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())

print('\nDecision Variables')
print('xf = ', model.xf.value)  
print('xl = ', model.xl.value)
print('yf = ', model.yf.value)  
print('yl = ', model.yl.value)
print('Pipeline distance: ', (model.obj())**0.5)
print('The point inside farm is : (', model.xf.value,',',model.yf.value,')')
print('The point inside lake is : (', model.xl.value,',',model.yl.value,')')

Solver status: ok
Solver termination condition: optimal

Objective =  41201.77861589313

Decision Variables
xf =  120.00000000068964
xl =  184.18861161997165
yf =  -39.99999999691607
yl =  152.56583486036618
Pipeline distance:  202.98221256034512
The point inside farm is : ( 120.00000000068964 , -39.99999999691607 )
The point inside lake is : ( 184.18861161997165 , 152.56583486036618 )


#Q-4

$Declaring \;two\; new\; decesion \;variables: \\
x_j = The\; x \;coordinate \;of\; the \;junction \\ 
y_j = The\; y\; coordinate\; of \;the\; junction \\
In\; this\; case,\; the \;objective \;function\; becomes: \\ 
{min} \sqrt[2]{(x_f-x_j)^2 + (y_f-y_j)^2 } +\sqrt[2]{(x_l-x_j)^2 + (y_l-y_j)^2 }+0.2\sqrt[2]{(x_j-15)^2 + (y_j-20)^2 }\\
\text{The constraints are:} \\ 
 (x_l-200)^2 +(y_l-200)^2 \leq 2500 \\ 
 (xf+3y_f)\leq0 \\ 
 0\leq x_f \leq150 \\ 
 -50 \leq y_f \leq 0  $

#Q-5

In [44]:
model.obj.deactivate()

model.xj =Var()
model.yj = Var()

model.obj_new = Objective(expr = ((model.xj-model.xf)**2 + (model.yj-model.yf)**2)**0.5+((model.xj-model.xl)**2+(model.yj-model.yl)**2)**0.5+(0.2*((model.xj-15)**2+(model.yj -20)**2)**0.5),sense=minimize)

model.pprint()

1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

6 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 120.00000000068964 :   150 : False : False :  Reals
    xj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   150 : 184.18861161997165 :   250 : False : False :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   -50 : -39.99999999691607 :     0 : False : False :  Reals
    yj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :

In [45]:
result = opt_ipopt.solve(model)
print(result)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 6
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.12616491317749023
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



#Q-6

In [46]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj_new())

print('\nDecision Variables')
print('xf = ', model.xf.value)  
print('xl = ', model.xl.value)
print('yf = ', model.yf.value)  
print('yl = ', model.yl.value)
print('xj = ', model.xj.value)  
print('yj = ', model.yj.value)
print('The point of junction is : (', model.xj.value,',',model.yj.value,')')
print('Reporting the locations of the two ends:')
print('The point inside farm is : (', model.xf.value,',',model.yf.value,')')
print('The point inside lake is : (', model.xl.value,',',model.yl.value,')')

Solver status: ok
Solver termination condition: optimal

Objective =  221.99510904190353

Decision Variables
xf =  74.46804648076933
xl =  175.0655597032054
yf =  -24.82268215956456
yl =  156.66094472839706
xj =  79.72453250966831
yj =  -9.053224072820475
The point of junction is : ( 79.72453250966831 , -9.053224072820475 )
Reporting the locations of the two ends:
The point inside farm is : ( 74.46804648076933 , -24.82268215956456 )
The point inside lake is : ( 175.0655597032054 , 156.66094472839706 )
